In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import tensorflow as tf
from tensorflow.contrib import slim
import torch
from torch import nn
import torch.nn.functional as F
import numpy as np
import os
from PIL import Image
from collections import OrderedDict
import torchvision.transforms as transforms
from old_generator import TorchGAN
from old_discriminator import TorchDiscriminator

In [3]:
# os.system("gdown --id 1TE1Bi5ym-TbLSnlkAAOP0sSQYkgTNl4A --output datasets/omniglot_data.npy")
# os.system("mkdir checkpoints")
# os.system("gdown --id 1qw_op6L2RebrGik0cBWquDK3RTculrVR --output checkpoints/model.ckpt.data-00000-of-00001")
# os.system("gdown --id 17UFRFwWOyJ_-tU72SH0o5mrg2-oEra95 --output checkpoints/model.ckpt.index")
# os.system("gdown --id 1iSOAjMUWLDemHljNHk0grCFdblSArBcP --output checkpoints/model.ckpt.meta")

In [4]:
raw_data = np.load("datasets/omniglot_data.npy")

In [5]:
curr_data = raw_data[1200]

In [6]:
from dagan_architectures import UResNetGenerator, Discriminator

d = Discriminator(batch_size=1, layer_sizes=[64, 64, 128, 128],
                        inner_layers=[5] * 4, name="discriminator")

In [7]:
inp1 = tf.placeholder(tf.float32, [1,28,28,1], 'inp1')
inp2 = tf.placeholder(tf.float32, [1,28,28,1], 'inp2')

In [8]:
result = d(inp1, inp2, training=False)


Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.


discr layers 22
discriminator_parameter_num 7072551


In [9]:
simple_inp = np.ones((1, 28, 28, 1))

In [10]:
checkpoint = "checkpoints/checkpoint.ckpt"
ckpt_reader = tf.train.load_checkpoint(checkpoint)
def read_tf(var):
    return torch.tensor(ckpt_reader.get_tensor(var))

def reshape_conv(tensor):
    return tensor.transpose(0, 3).transpose(1, 2).transpose(2, 3)

In [19]:
# Run tf model

fine_tune = slim.assign_from_checkpoint_fn(
    checkpoint,
    tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES))

sess = tf.Session()
fine_tune(sess)

final_results = sess.run(
    result,
    feed_dict={inp1: curr_data[0:1], inp2: curr_data[1:2]},
)
np.array(final_results[0])

INFO:tensorflow:Restoring parameters from checkpoints/checkpoint.ckpt


array([[-191.75293]], dtype=float32)

In [12]:
torch_d = TorchDiscriminator(28, 1, dropout_rate=0.1)
torch_d.eval()
None

In [13]:
# Seed torch generator weights

def convert_conv_layer(torch_layer, conv_path, norm_path=None):
    if norm_path is not None:
        torch_layer.norm.weight.data = read_tf(norm_path + "/gamma").unsqueeze(-1).unsqueeze(-1)
        torch_layer.norm.bias.data = read_tf(norm_path + "/beta").unsqueeze(-1).unsqueeze(-1)
    
    torch_layer.conv.weight.data = reshape_conv(read_tf(conv_path + "/kernel"))
    torch_layer.conv.bias.data = read_tf(conv_path + "/bias")
    
num_layers = torch_d.num_inner_layers

# Encoders
convert_conv_layer(
    torch_d.encode0, "discriminator/conv_layers/g_conv0/conv2d", "discriminator/conv_layers/g_conv0/LayerNorm")

for i in range(1, torch_d.depth):
    module = torch_d._modules["encode%d" % i]
    convert_conv_layer(
        module.pre_conv,
        f"discriminator/conv_layers/g_conv{i}/conv2d"
    )

    for j in range(num_layers + 1):
        norm_suffix = f"_{j}" if j > 0 else ""
        convert_conv_layer(
            module._modules[f"conv{j}"],
            f"discriminator/conv_layers/g_conv{i}/conv2d_{j + 1}",
            f"discriminator/conv_layers/g_conv{i}/LayerNorm{norm_suffix}",
        )

torch_d.dense1.weight.data = read_tf("discriminator/discriminator_dense_block/dense/kernel").T
torch_d.dense1.bias.data = read_tf("discriminator/discriminator_dense_block/dense/bias").T
torch_d.dense2.weight.data = read_tf("discriminator/discriminator_out_block/outputs/kernel").T
torch_d.dense2.bias.data = read_tf("discriminator/discriminator_out_block/outputs/bias").T

In [22]:
def to_torch(arr):
    return torch.tensor(arr).transpose(0, 1).transpose(0, 2).unsqueeze(0)
x1 = to_torch(curr_data[0])
x2 = to_torch(curr_data[1])

In [23]:
out = torch_d(x1, x2)

In [24]:
out

tensor([[-191.7530]], grad_fn=<AddmmBackward>)

In [ ]:
def render_image_arr(arr):
    arr = np.uint8(arr * 256)
    arr = arr.reshape(arr.shape[:-1])
    display(Image.fromarray(arr, mode='L').resize((224, 224)))
    
def render_torch(arr):
    arr = np.uint8(arr * 256)
    display(Image.fromarray(arr, mode='L').resize((224, 224)))

In [ ]:
# Render tf
render_image_arr(np.array(final_results[0][0]) * 0.5 + 0.5)

In [ ]:
# Render torch
refined_torch_out = torch_out.detach().numpy() * 0.5 + 0.5

render_torch(refined_torch_out[0][0])

In [ ]:
tf.train.list_variables(checkpoint)

In [ ]:
read_tf('discriminator/conv_layers/g_conv0/LayerNorm/beta')

In [ ]:
var = tf.trainable_variables()[-2]
var = tf.assign(var, [1., 2., 3.])
sess.run(var)
# var.eval(sess)

In [ ]:
tf.trainable_variables()[-2].assign([1,2,3],sess)